In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [4]:
%%sql
SELECT *
  FROM facts
LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [9]:
%%sql
SELECT MIN(population) as Min_Population,
       MAX(population) as Max_Population,
       MIN(population_growth) as Min_Pop_Growth,
       MAX(population_growth) as Max_Pop_Growth
FROM facts

Done.


Min_Population,Max_Population,Min_Pop_Growth,Max_Pop_Growth
0,7256490011,0.0,4.02


Interesting - min population of 0, and max population greater than 7.2bn.

Writing a query to return countries with min population:

In [7]:
%%sql
SELECT *
FROM facts
WHERE population == (SELECT min(population)
                     FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


Writing a query to return countries with max population:

In [8]:
%%sql
SELECT *
FROM facts
WHERE population == (SELECT MAX(population)
                    FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


Table 'facts' in the database includes a row for 'Total World'.

Recalculating statistics above excluding whole world row:

In [11]:
%%sql
SELECT MIN(population) as Min_Population,
       MAX(population) as Max_Population,
       MIN(population_growth) as Min_Pop_Growth,
       MAX(population_growth) as Max_Pop_Growth
FROM facts
WHERE name <> 'World'

Done.


Min_Population,Max_Population,Min_Pop_Growth,Max_Pop_Growth
0,1367485388,0.0,4.02


### Reviewing Population Density

Calculating average of 'population' and 'area' columns to review population density, and exluding 'total world' row:

In [18]:
%%sql
SELECT AVG(population) AS avg_pop, AVG(area) AS avg_area
FROM facts
WHERE name <> 'World'

Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


Identifying countries that are densely populated:

- above-average values for population
- below-average values for area

In [20]:
%%sql
SELECT *
FROM facts
WHERE population > (SELECT AVG(population)
                    FROM facts
                    WHERE name <> 'World')
    AND area < (SELECT AVG(area)
               FROM facts
               WHERE name <> 'World')

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


### Next Steps

##### Which country has the most people? Which country has the highest growth rate?

In [22]:
%%sql
SELECT name, MAX(population) AS max_population
FROM facts
WHERE name <> 'World'

Done.


name,max_population
China,1367485388


In [23]:
%%sql
SELECT name, MAX(population_growth) AS max_growth
FROM facts
WHERE name <> 'World'

Done.


name,max_growth
South Sudan,4.02


##### Which countries have the highest ratios of water to land? Which countries have more water than land?

In [38]:
%%sql
SELECT name, ROUND(CAST(area_water/area_land AS FLOAT),4) AS water_land_ratio
FROM facts
WHERE name <> 'World'
ORDER BY water_land_ratio DESC
LIMIT 5

Done.


name,water_land_ratio
British Indian Ocean Territory,905.0
Virgin Islands,4.0
Afghanistan,0.0
Albania,0.0
Algeria,0.0


In [39]:
%%sql
SELECT *
FROM facts
WHERE area_water > area_land

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67


##### Which countries will add the most people to their populations next year?

In [43]:
%%sql
SELECT name, ROUND(CAST(population*population_growth AS FLOAT),2) AS population_add
FROM facts
WHERE name <> 'World'
ORDER BY population_add desc
LIMIT 5

Done.


name,population_add
India,1527068612.48
China,615368424.6
Nigeria,444827037.2
Pakistan,290665336.62
Ethiopia,287456216.91


##### Which countries have a higher death rate than birth rate?


In [45]:
%%sql
SELECT *
FROM facts
WHERE death_rate > birth_rate
 AND name <> 'World'

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7
22,bk,Bosnia and Herzegovina,51197,51187,10,3867055,0.13,8.87,9.75,0.38
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
44,hr,Croatia,56594,55974,620,4464844,0.13,9.45,12.18,1.39
47,ez,Czech Republic,78867,77247,1620,10644842,0.16,9.63,10.34,2.33
57,en,Estonia,45228,42388,2840,1265420,0.55,10.51,12.4,3.6
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
67,gr,Greece,131957,130647,1310,10775643,0.01,8.66,11.09,2.32
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33


##### Which countries have the highest population/area ratio, and how does it compare to list we found in the review of the population density?

In [48]:
%%sql
SELECT name, population, population/area AS pop_area
FROM facts
WHERE name <> 'World'
ORDER BY pop_area DESC

Done.


name,population,pop_area
Macau,592731,21168
Monaco,30535,15267
Singapore,5674472,8141
Hong Kong,7141106,6445
Gaza Strip,1869055,5191
Gibraltar,29258,4876
Bahrain,1346613,1771
Maldives,393253,1319
Malta,413965,1310
Bermuda,70196,1299
